In [2]:
import tensorflow as tf
from keras import __version__
tf.keras.__version__ = __version__

import time
import random
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Embedding, Reshape

from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

import gym

from env_class import BatteryManagementEnv

plt.style.use("ggplot")

In [3]:
env = BatteryManagementEnv()
env.reset()

c:\Users\janle\AppData\Local\Programs\Python\Python311\Lib\site-packages\gym\spaces\box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


array([50.], dtype=float32)

In [4]:
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

Action Space MultiDiscrete([3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3])
State Space Box(0.0, 100.0, (1,), float32)


In [5]:
from keras.layers import Dense, Input
from keras.models import Model

# Assuming the observation space includes the last 24 actions
input_shape = (1,)  # Current charge level
model_input = Input(shape=(input_shape))
x = Flatten()(model_input)
x = Dense(64, activation='relu')(x)
output = Dense(24 * 3, activation='linear')(x)  # Each action has 3 possible values
model_output = Reshape((24, 3))(output)  # Reshaping it to (24, 3)

model = Model(inputs=model_input, outputs=model_output)

In [6]:
import tensorflow as tf
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.networks import q_network
from tf_agents.agents.dqn import dqn_agent
from tf_agents.utils import common
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.policies import random_tf_policy
from tf_agents.metrics import tf_metrics
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import utils

import gym

try:
    env_name = 'BatteryManagementEnv'
    env = suite_gym.load(env_name)
    print(env)
    print(whatever)
    tf_env = tf_py_environment.TFPyEnvironment(env)
    print("Registration successful!")
except gym.error.UnregisteredEnv:
    print("Environment registration failed.")
tf_env = tf_py_environment.TFPyEnvironment(env)

# Define the Q-Network
fc_layer_params = (100, 50)
q_net = q_network.QNetwork(
    tf_env.observation_spec(),
    tf_env.action_spec(),
    fc_layer_params=fc_layer_params)

# Create the DQN Agent
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=1e-3)
train_step_counter = tf.Variable(0)

agent = dqn_agent.DqnAgent(
    tf_env.time_step_spec(),
    tf_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent.initialize()

# Data Collection
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=tf_env.batch_size,
    max_length=100000)

# Create a driver to collect experience
collect_driver = dynamic_step_driver.DynamicStepDriver(
    tf_env,
    agent.collect_policy,
    observers=[replay_buffer.add_batch],
    num_steps=1)  # Collect 1 step each time the driver is called

# (Optional) Initial data collection with a random policy
random_policy = random_tf_policy.RandomTFPolicy(tf_env.time_step_spec(), tf_env.action_spec())
init_driver = dynamic_step_driver.DynamicStepDriver(
    tf_env,
    random_policy,
    observers=[replay_buffer.add_batch],
    num_steps=1)  # Collect 1 step each time the driver is called
final_time_step, final_policy_state = init_driver.run()

# Training the DQN Agent
# Define a function for the training loop here...

# Metrics and Evaluation
# Define metrics for evaluation...


ValueError: too many values to unpack (expected 2)
  In call to configurable 'load' (<function load at 0x00000296ED7EA020>)

In [ ]:
dqn.fit(env, nb_steps=1000, visualize=False, verbose=1, nb_max_episode_steps=200, log_interval=10000)

Training for 1000 steps ...
Interval 1 (0 steps performed)
  117/10000 [..............................] - ETA: 9s - reward: -2.6923

c:\Users\janle\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


  991/10000 [=>............................] - ETA: 30s - reward: -1.1362done, took 3.465 seconds
